Загрузим датасет

In [1]:
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

# Замена имени колонок
df = df.rename(columns={
    "sepal length (cm)": "длина чашелистика (см)",
    "sepal width (cm)": "ширина чашелистика (см)",
    "petal length (cm)": "длина лепестка (см)",
    "petal width (cm)": "ширина лепестка (см)",
    "target": "класс"
})

# Отобразим примеры
df.head()

,длина чашелистика (см),ширина чашелистика (см),длина лепестка (см),ширина лепестка (см),класс
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


делим на test и train

In [2]:
from sklearn.model_selection import train_test_split
X, y = df.drop('класс', axis=1), df['класс']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Создаем пайплайн

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

# Пайплайн с масштабированием
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # масштабирование
    ('classifier', DecisionTreeClassifier(random_state=42))  # модель
])

# Просмотр пайплайна
pipeline

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', DecisionTreeClassifier(random_state=42))])

Параметры для перебора для методов:

* Grid Search
* Random Search

In [4]:
# Параметры
param_grid = {
    "classifier__criterion": ["gini", "entropy"],
    "classifier__max_depth": [3, 4, 5, 6, 7, None],
    "classifier__min_samples_split": [2, 5, 10],
    "classifier__min_samples_leaf": [1, 2, 4],
    "classifier__max_features": ["sqrt", "log2", None]
}

# Grid Search
- Перебирает все возможные комбинации указанных гиперпараметров.
- Для каждой комбинации обучает модель и проверяет качество на кросс-валидации.
- Возвращает лучшую комбинацию по заданной метрике (например, accuracy).

In [5]:
from sklearn.model_selection import GridSearchCV

# Создание сетки
gs_tree = GridSearchCV(
    estimator=pipeline,     # модель
    param_grid=param_grid,  # параметры
    scoring="accuracy",     # метрика
    cv=5,                   # K-folds для кросс валидации
    n_jobs=-1,              # параллель на всю катушку
    verbose=3,              # кол-во информации для отображения
    refit="accuracy"
)

# Обучение моделей
gs_tree.fit(X_train, y_train)

# Результаты
print("\nЛучшие параметры:")
for key, value in gs_tree.best_params_.items():
    print(f'{key} -> {value}')
print("\nЛучшее качество на кросс-валидации:", gs_tree.best_score_)
print("Точность на тесте:", gs_tree.score(X_test, y_test))

Fitting 5 folds for each of 324 candidates, totalling 1620 fits

Лучшие параметры:
classifier__criterion -> entropy
classifier__max_depth -> 5
classifier__max_features -> None
classifier__min_samples_leaf -> 4
classifier__min_samples_split -> 2

Лучшее качество на кросс-валидации: 0.9583333333333334
Точность на тесте: 1.0


# Random Search

* **Идея:** выбираем случайные комбинации гиперпараметров из заранее определённых диапазонов.
* **Плюсы:**
  * Прост в реализации.
  * Эффективнее, чем Grid Search для моделей с большим числом гиперпараметров.
  * Хорош для грубой оценки влияния гиперпараметров.
* **Минусы:**
  * Не гарантирует нахождение оптимального решения.
  * Требует большого числа итераций при сложных пространствах гиперпараметров.


In [6]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=pipeline,                 # модель
    param_distributions=param_grid,     # параметры
    n_iter=40,                          # кол-во попыток (кандидатов)
    scoring='accuracy',                 # метрика
    cv=5,                               # K-folds для кросс валидации
    n_jobs=-1,                          # параллель на всю катушку
    verbose=3                           # кол-во информации для отображения
)

random_search.fit(X_train, y_train)

# Результаты
print("\nЛучшие параметры:")
for key, value in random_search.best_params_.items():
    print(f'{key} -> {value}')
print("\nЛучшее качество на кросс-валидации:", random_search.best_score_)
print("Точность на тесте:", random_search.score(X_test, y_test))

Fitting 5 folds for each of 40 candidates, totalling 200 fits

Лучшие параметры:
classifier__min_samples_split -> 5
classifier__min_samples_leaf -> 4
classifier__max_features -> None
classifier__max_depth -> 7
classifier__criterion -> entropy

Лучшее качество на кросс-валидации: 0.9583333333333334
Точность на тесте: 1.0


# Байесовская оптимизация с Optuna

* **Идея:** строим вероятностную модель функции потерь и используем её, чтобы выбирать новые гиперпараметры «умнее», чем случайно.
* **Плюсы:**
  * Эффективнее Random Search при сложных пространствах гиперпараметров.
  * Балансирует *exploration* (исследование) и *exploitation* (использование лучших областей).
* **Минусы:**
  * Сложнее для понимания и настройки.
  * Требует больше зависимостей (Optuna, иногда PyTorch/Scikit-learn).

[Подробнее](https://optuna.org/)

In [7]:
# Установка Optuna
!pip install optuna

In [8]:
import optuna
from sklearn.model_selection import cross_val_score

def objective_dt(trial):
    # Настройка гиперпараметров для Decision Tree
    max_depth = trial.suggest_int('classifier__max_depth', 1, 20)
    min_samples_split = trial.suggest_int('classifier__min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('classifier__min_samples_leaf', 1, 10)
    criterion = trial.suggest_categorical('classifier__criterion', ['gini', 'entropy'])

    pipeline.set_params(
        classifier__max_depth=max_depth,
        classifier__min_samples_split=min_samples_split,
        classifier__min_samples_leaf=min_samples_leaf,
        classifier__criterion=criterion
    )

    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy').mean()
    return score


# Создаём исследование
study_dt = optuna.create_study(
    direction='maximize'  # максимизирукм cross_val_score чтобы он был наибольшим
)

study_dt.optimize(
    objective_dt,
    n_trials=60,  # количество попыток
    n_jobs=-1,
)


# Лучшие параметры
print("\nЛучшие параметры Decision Tree:")
for key, value in study_dt.best_params.items():
    print(f'{key} -> {value}')

pipeline.set_params(**study_dt.best_params)
pipeline.fit(X_train, y_train)
print("\nЛучшее качество на кросс-валидации:", cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy').mean())
print("Точность Decision Tree на тесте:", pipeline.score(X_test, y_test))


[I 2025-11-09 13:12:34,426] A new study created in memory with name: no-name-073359e1-ed73-4631-b8ee-b0b9c736bf25
[I 2025-11-09 13:12:34,986] Trial 1 finished with value: 0.9166666666666667 and parameters: {'classifier__max_depth': 18, 'classifier__min_samples_split': 3, 'classifier__min_samples_leaf': 10, 'classifier__criterion': 'entropy'}. Best is trial 1 with value: 0.9166666666666667.
[I 2025-11-09 13:12:34,998] Trial 0 finished with value: 0.9166666666666667 and parameters: {'classifier__max_depth': 12, 'classifier__min_samples_split': 4, 'classifier__min_samples_leaf': 8, 'classifier__criterion': 'entropy'}. Best is trial 1 with value: 0.9166666666666667.
[I 2025-11-09 13:12:35,183] Trial 2 finished with value: 0.9166666666666667 and parameters: {'classifier__max_depth': 13, 'classifier__min_samples_split': 8, 'classifier__min_samples_leaf': 2, 'classifier__criterion': 'entropy'}. Best is trial 1 with value: 0.9166666666666667.
[I 2025-11-09 13:12:35,202] Trial 3 finished with v


Лучшие параметры Decision Tree:
classifier__max_depth -> 12
classifier__min_samples_split -> 11
classifier__min_samples_leaf -> 5
classifier__criterion -> entropy

Лучшее качество на кросс-валидации: 0.95
Точность Decision Tree на тесте: 1.0


# TPOT - Automated Machine Learning (AutoML) инструмент

TPOT (Tree-based Pipeline Optimization Tool) - это библиотека автоматизированного машинного обучения (AutoML), которая использует генетические алгоритмы для оптимизации полных конвейеров машинного обучения. В отличие от Optuna и RandomizedSearchCV, которые оптимизируют только гиперпараметры существующей модели, TPOT автоматически:


* Выбирает лучшие алгоритмы машинного обучения
* Создает оптимальные preprocessing steps
* Настраивает гиперпараметры
* Строит полные конвейеры обработки данных

## Ключевые особенности TPOT

### 1. **Генетическое программирование**
TPOT использует эволюционные алгоритмы для "размножения" и "мутации" конвейеров, постепенно улучшая их качество.

### 2. **Автоматический подбор моделей**
Может выбирать между различными классификаторами/регрессорами:
- Decision Trees, Random Forest, Gradient Boosting
- SVM, KNN, Logistic Regression
- Neural Networks и другие

### 3. **Предобработка данных**
Автоматически добавляет необходимые этапы:
- StandardScaler, MinMaxScaler, RobustScaler
- PCA, SelectPercentile, VarianceThreshold
- OneHotEncoding, PolynomialFeatures

## Преимущества TPOT

- **Экономия времени** - автоматизирует рутинные задачи
- **Обнаружение неочевидных решений** - может найти нестандартные комбинации методов
- **Воспроизводимость** - генерирует готовый Python код
- **Избегает переобучения** - использует кросс-валидацию

[Подробнее](https://github.com/EpistasisLab/tpot?ysclid=mhrqdj7la9508680297)

In [9]:
!pip install tpot

In [21]:
from tpot import TPOTClassifier
from sklearn.metrics import accuracy_score


# Создаём TPOT-классификатор
tpot = TPOTClassifier(
    generations=5,        # количество поколений эволюций
    population_size=20,   # размер популяции
    verbose=2,            # уровень детализации вывода
    random_state=42,
    cv=5,                 # количество фолдов кросс-валидации
)


In [22]:
# Обучаем
print("Начало оптимизации с TPOT...")
tpot.fit(X_train, y_train)


/usr/local/lib/python3.12/dist-packages/tpot/tpot_estimator/estimator.py:458: UserWarning: Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.
  warnings.warn("Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.")
/usr/local/lib/python3.12/dist-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33151 instead
  warnings.warn(
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:34845
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:33151/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:37921'


Начало оптимизации с TPOT...


INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:43941 name: 0
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:43941
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:55096
INFO:distributed.scheduler:Receive client connection: Client-10da9636-bd70-11f0-953f-0242ac1c000c
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:55098

Generation:   0%|          | 0/5 [01:31<?, ?it/s]

Generation:  80%|████████  | 4/5 [03:23<00:49, 49.13s/it]INFO:distributed.nanny:Closing Nanny gracefully at 'tcp://127.0.0.1:37921'. Reason: worker-close
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:55096; closing.
INFO:distributed.scheduler:Remove worker addr: tcp://127.0.0.1:43941 name: 0 (stimulus_id='handle-worker-cleanup-1762695186.6772058')
ERROR:distributed.scheduler:Removing worker 'tcp://127.0.0.1:43941' caused the cluster to lose scattered data, which can't be recovered: {'Series-c5c1a8173e73d18

TPOTClassifier(cv=5, random_state=42,
               search_space=<tpot.search_spaces.pipelines.sequential.SequentialPipeline object at 0x7ac49bd5f710>)

In [23]:
# Оценка качества
y_pred = tpot.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Точность на тестовой выборке: {test_accuracy:.4f}")

# Лучшие параметры и конвейер
print("\nЛучший конвейер:")
print(tpot.fitted_pipeline_)

Точность на тестовой выборке: 1.0000

Лучший конвейер:
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('selectpercentile',
                 SelectPercentile(percentile=37.1143747020325)),
                ('featureunion-1',
                 FeatureUnion(transformer_list=[('skiptransformer',
                                                 SkipTransformer()),
                                                ('passthrough',
                                                 Passthrough())])),
                ('featureunion-2',
                 FeatureUnion(transformer_list=[('skiptransformer',
                                                 SkipTransformer()),
                                                ('passthrough',
                                                 Passthrough())])),
                ('quadraticdiscriminantanalysis',
                 QuadraticDiscriminantAnalysis(reg_param=0.0196113874995))])


# Сравнение

| Метод | Что оптимизирует | Автоматизация | Время | Качество |
|-------|------------------|---------------|-------|----------|
| **Optuna** | Гиперпараметры модели | Средняя | Среднее | Высокое |
| **RandomizedSearchCV** | Гиперпараметры модели | Низкая | Быстрое | Среднее |
| **TPOT** | Полный конвейер + модели | Полная | Длительное | Очень высокое |